In [354]:
from qoop.core import state, ansatz, metric
import qiskit
from qiskit import transpile
from qiskit.quantum_info import Operator, DensityMatrix, Kraus
from scipy.linalg import qr
import numpy as np
import tensorflow as tf

In [355]:
def create_kraus_operators(unitary_matrix):
    '''
        Create a set of Kraus Operators from the input unitary matrix, using QR decomposition
    '''
    kraus_ops = []
    Q, R = qr(unitary_matrix)

    #Q: a 2^N x 2^N matrix, N is the number of qubits
    for q in Q:
        q = np.expand_dims(q, 1)
        kraus_ops.append(q @ np.transpose(np.conjugate(q)))
    return tf.convert_to_tensor(kraus_ops)

def create_unitary_matrix(num_qubits: int):
    """
    Generate a random unitary matrix of size 2^n x 2^n.
    """
    dimension = 2 ** num_qubits
    # Generate a random complex matrix
    random_matrix = np.random.normal(size=(dimension, dimension)) + 1j * np.random.normal(size=(dimension, dimension))
    # Perform QR decomposition
    q, _ = qr(random_matrix)
    return q

def calculate_rho2_dephasing(input_rho, num_qubits: int, gamma: float): #for verification
    '''
    Calculate rho2 by dephasing
    '''
    
    # Convert DensityMatrix to numpy array
    rho = input_rho.data
    # Define the Pauli-Z matrix
    sigma_z = np.array([[1, 0], [0, -1]])

    # Calculate the factors
    alpha = 1 + np.sqrt(1 - gamma)
    beta = 1 - np.sqrt(1 - gamma)
    
    # n qubits => qubit thứ n => I @ I @ .... sigma_z (n) @....I
    # Loop for multiple qubits
    for i in range(num_qubits):
        # Create the tensor product of Pauli-Z matrices for all qubits
        sigma_z_i = np.eye(1)
        for j in range(num_qubits):
            if j == i:
                sigma_z_i = np.kron(sigma_z_i, sigma_z)
            else:
                sigma_z_i = np.kron(sigma_z_i, np.eye(2))
        
        # Apply the dephasing formula
        rho = alpha * rho + beta * (sigma_z_i @ rho @ sigma_z_i)
        # Normalize the density matrix (optional, depending on the context)
        rho /= np.trace(rho)

    return rho

def calculate_rho2_from_unitary(rho, unitary_matrix):
    '''
    Calculate rho' by applying U @ rho @ U(dagger)
    '''
    rho_2 = unitary_matrix  @ rho.data @ np.transpose(np.conjugate(unitary_matrix))

    return rho_2

def calculate_rho_3 (rho_2, kraus_operators):
    '''
        sum(K @ rho_2 @ K(dagger)) //rho" = rho
    '''

    rho_3 = sum(K @ rho_2 @ np.transpose(np.conjugate(K)) for K in kraus_operators)
    return rho_3





In [356]:
def compilation_trace_fidelity(rho, sigma):
    """Calculating the fidelity metric

    Args:
        - rho (DensityMatrix): first density matrix
        - sigma (DensityMatrix): second density matrix

    Returns:
        - float: trace metric has value from 0 to 1
    """
    rho_2 = tf.linalg.sqrtm((tf.linalg.sqrtm(rho)) @ (rho))

    # Cast to a supported type
    real_part = tf.math.real(rho_2)
    imaginary_part = tf.math.imag(rho_2)

    # Check for NaNs in both real and imaginary parts
    contains_nan_real = tf.reduce_any(tf.math.is_nan(real_part))
    contains_nan_imag = tf.reduce_any(tf.math.is_nan(imaginary_part))

    contains_nan = contains_nan_real or contains_nan_imag

    if contains_nan == True:
        rho_2 = rho

    return tf.linalg.trace(
            rho_2
            @ (tf.linalg.sqrtm(sigma))
        )
def frobenius_norm(rho, sigma):
    """
    Compute the Frobenius norm between two matrices.

    Parameters:
    rho (numpy.ndarray): The first matrix.
    sigma (numpy.ndarray): The second matrix.

    Returns:
    float: The Frobenius norm between rho and rho_prime.
    """

    # Ensure the matrices are TensorFlow tensors
    rho = tf.convert_to_tensor(rho, dtype=tf.complex128)
    sigma = tf.convert_to_tensor(sigma, dtype=tf.complex128)

    # Compute the difference between the matrices
    diff = rho - sigma

    # Compute the Frobenius norm
    #norm = tf.linalg.normalize(diff, ord='fro')
    norm = tf.sqrt(tf.reduce_sum(tf.square(diff)))
    return norm

#cost func to compare 2 given rhos
def cost(rho, rho_3):
    #return tf.square(frobenius_norm(rho, rho_3))
    return 1 - compilation_trace_fidelity(rho, rho_3)

In [357]:
#Auto Diff
'''
    rho: The initial rho
    unitary_matrix: randomly generated circle in initialization step
    kraus_operators: set of kraus operators
    n: number of qubits
    alpha: learning rate (?)'''
def calculate_derivative(rho, rho2, kraus_operators, num_qubits, alpha=0.1):
    tensorKraus = tf.Variable(kraus_operators)
    with tf.GradientTape() as tape:
        rho3 = calculate_rho_3(rho2, tensorKraus)
        f = cost(rho, rho3) 
    
    # Get the gradient of y with respect to x
    c = tape.gradient(f, tensorKraus)

    # Reshape c to kN * N matrix
    c = tf.reshape(c, (2**num_qubits * 2**num_qubits,2**num_qubits))

    # Reshape kraus_operators to kN * N matrix
    kraus_operators = tf.reshape(kraus_operators, (2**num_qubits * 2**num_qubits,2**num_qubits))
    
    # Compute the projection term
    proj = c - kraus_operators @ (np.transpose(np.conjugate(c)) @ kraus_operators  + np.transpose(np.conjugate(kraus_operators)) @ c) / 2

    # Update the Kraus operators
    updated_kraus_operators = kraus_operators - alpha * proj
    
    # Return the updated Kraus operators
    return updated_kraus_operators



In [358]:
def optimize(rho, rho2, kraus_operators, num_qubits, alpha=0.1, num_loop = 1000):
    kraus_operators_copy = tf.identity(kraus_operators)
    # try looping manually
    for i in range (0, num_loop):
        rho3 = calculate_rho_3(rho2, kraus_operators_copy)
        _cost = cost(rho, rho3) 

        #Update Kraus Operators
        kraus_operators_copy=calculate_derivative(rho, rho2, kraus_operators_copy, num_qubits, alpha)

        #Reshape
        kraus_operators_copy = tf.reshape(kraus_operators_copy, (2**num_qubits,2**num_qubits,2**num_qubits))
    
        #print('K(t)K\n',sum(K @ np.transpose(np.conjugate(K)) for K in KrausOperatorsTry)) # I

        print (_cost)
    return kraus_operators_copy

In [359]:
def print_check_unitary(unitary_matrix):
    print("U @ U(dagger)")
    print(unitary_matrix @ np.transpose(np.conjugate(unitary_matrix)))
        
def print_check_kraus(kraus_operators):
    print("sum( K @ K(dagger) )")
    print(sum(K @ np.transpose(np.conjugate(K)) for K in kraus_operators))

In [360]:
def V(num_qubits: int):
    '''
        Create a ansatz V with n qubits and assign its parameters
    '''
    #Assign random parameter
    circuit = ansatz.stargraph(num_qubits=num_qubits)
    num_params = circuit.num_parameters
    x0 = 2 * np.pi * np.random.random(num_params)
    circuit = circuit.assign_parameters(dict(zip(circuit.parameters, x0)))
    return circuit

In [361]:
num_qubits = 2

# Initial state |0⟩ density matrix for multiple qubits
rho = DensityMatrix.from_label('0' * num_qubits)

# Generate a random unitary_matrix and random set of kraus_operators
#unitary_matrix = create_unitary_matrix(num_qubits=num_qubits)
circuit = V(num_qubits)
# Get the unitary operator corresponding to the circuit
unitary_op = Operator(circuit)
# Get the unitary matrix
unitary_matrix = unitary_op.data
# Get first set of kraus_operators
kraus_operators = create_kraus_operators(unitary_matrix=unitary_matrix) 

#print_check_unitary(unitary_matrix=unitary_matrix)
#print_check_kraus(kraus_operators=kraus_operators)

rho2 = calculate_rho2_dephasing(rho, num_qubits, 0.3)


In [362]:
# Optimizing
out_kraus_operators = optimize(rho, rho2, kraus_operators, num_qubits, 0.1, 1000)

tf.Tensor((0.49830669807294936+0j), shape=(), dtype=complex128)
tf.Tensor((0.4608515438676004+0j), shape=(), dtype=complex128)
tf.Tensor((0.42535338365857434+0j), shape=(), dtype=complex128)
tf.Tensor((0.3918390784611615+0j), shape=(), dtype=complex128)
tf.Tensor((0.3603130481498099+0j), shape=(), dtype=complex128)
tf.Tensor((0.33075952423686084+0j), shape=(), dtype=complex128)
tf.Tensor((0.3031450335822271+0j), shape=(), dtype=complex128)
tf.Tensor((0.27742099403302056+0j), shape=(), dtype=complex128)
tf.Tensor((0.2535263174585344+0j), shape=(), dtype=complex128)
tf.Tensor((0.23138993339817837+0j), shape=(), dtype=complex128)
tf.Tensor((0.21093316564429698+0j), shape=(), dtype=complex128)
tf.Tensor((0.19207191294400094+0j), shape=(), dtype=complex128)
tf.Tensor((0.17471860243721093+0j), shape=(), dtype=complex128)
tf.Tensor((0.15878389967403905+0j), shape=(), dtype=complex128)
tf.Tensor((0.1441781716647077+0j), shape=(), dtype=complex128)
tf.Tensor((0.13081270930113997+0j), shape=(), 

In [363]:
print(1 - cost(rho, calculate_rho_3(rho2, out_kraus_operators)))

#K(dagger)

tf.Tensor((1.0000000000000002+0j), shape=(), dtype=complex128)


In [364]:
print(calculate_rho_3(rho2, out_kraus_operators))

tf.Tensor(
[[ 1.00000000e+00+0.j  1.28616040e-16+0.j -1.31656262e-16+0.j
  -1.09165996e-16+0.j]
 [ 1.28616040e-16+0.j  1.63344651e-31+0.j -1.18790105e-31+0.j
  -1.10761184e-31+0.j]
 [-1.31656262e-16+0.j -1.18790105e-31+0.j  1.59461077e-31+0.j
   1.11822637e-31+0.j]
 [-1.09165996e-16+0.j -1.10761184e-31+0.j  1.11822637e-31+0.j
   1.98242658e-31+0.j]], shape=(4, 4), dtype=complex128)


In [365]:
print(out_kraus_operators)

tf.Tensor(
[[[ 4.97014282e-01+0.j  2.77008580e-01+0.j  2.21489539e-01+0.j
    2.49145000e-01+0.j]
  [-1.80304134e-16+0.j  2.87406982e-01+0.j  2.46321120e-01+0.j
    2.66850785e-01+0.j]
  [-3.14904654e-17+0.j  2.51181224e-01+0.j  2.15396637e-01+0.j
    2.33276761e-01+0.j]
  [-5.00841759e-17+0.j  2.69267434e-01+0.j  2.30838560e-01+0.j
    2.50040280e-01+0.j]]

 [[ 5.44145716e-01+0.j -2.52936768e-01+0.j  2.54873622e-01+0.j
   -2.48567687e-01+0.j]
  [ 8.36423308e-17+0.j  2.45228135e-01+0.j -2.46461760e-01+0.j
    2.32098242e-01+0.j]
  [-8.28506075e-17+0.j -2.46289157e-01+0.j  2.47528301e-01+0.j
   -2.33104931e-01+0.j]
  [ 2.08508081e-16+0.j  2.29742412e-01+0.j -2.30900714e-01+0.j
    2.17476705e-01+0.j]]

 [[ 4.54966547e-01+0.j  2.43511419e-01+0.j -2.41865086e-01+0.j
   -2.48627185e-01+0.j]
  [ 3.50937877e-16+0.j  2.52395684e-01+0.j -2.51165154e-01+0.j
   -2.66669128e-01+0.j]
  [-3.47929159e-16+0.j -2.51303552e-01+0.j  2.50078461e-01+0.j
    2.65517446e-01+0.j]
  [-3.87878554e-16+0.j -2.69